In [1]:
import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM
from ast import literal_eval
from sklearn.model_selection import train_test_split

import os
import random
import numpy as np
import pandas as pd
import json
from tqdm import tqdm

In [2]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

set_seed(42) # magic number :)

In [3]:
ROOT_DIR = '/data/ephemeral/pro-nlp-generationfornlp-nlp-13'
DATA_DIR = os.path.join(ROOT_DIR, 'data')

In [4]:
dataset = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             2031 non-null   object 
 1   paragraph      2031 non-null   object 
 2   problems       2031 non-null   object 
 3   question_plus  0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 63.6+ KB


In [5]:
def flatten_df(
    df,
    problems_col="problems",
):
    records = []

    for _, row in df.iterrows():
        problems = literal_eval(row[problems_col])

        record = {
            "id": row["id"],
            "paragraph": row["paragraph"],
            "question": problems["question"],
            "choices": problems["choices"],
            "answer": problems.get("answer", None),
            "question_plus": problems.get("question_plus", None),
        }

        records.append(record)

    df = pd.DataFrame(records)


    return df

In [6]:
df = flatten_df(dataset)

In [7]:
df["choices_len"] = df["choices"].apply(len)
df['choices_len'].value_counts(dropna=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             2031 non-null   object
 1   paragraph      2031 non-null   object
 2   question       2031 non-null   object
 3   choices        2031 non-null   object
 4   answer         2031 non-null   int64 
 5   question_plus  0 non-null      object
 6   choices_len    2031 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 111.2+ KB


In [8]:
df4 = df[df.choices_len == 4]
df5 = df[df.choices_len == 5]

In [58]:
df4.answer.value_counts()

answer
4    221
3    203
2    185
1    183
Name: count, dtype: int64

In [9]:
df5.answer.value_counts()

answer
1    765
2    265
3    116
4     62
5     31
Name: count, dtype: int64

In [10]:
def shuffle_choices(choices, answer, seed=42):
    idx = answer - 1
    zipped = list(enumerate(choices))
    
    rng = random.Random(seed)  # 시드 고정
    rng.shuffle(zipped)

    new_choices = [c for _, c in zipped]
    new_answer = [i for i, (orig_i, _) in enumerate(zipped) if orig_i == idx][0] + 1

    return new_choices, new_answer

# 시드 고정
seed = 42
shuffled_df5 = [
    shuffle_choices(c, a, seed + i)  # i를 더해서 각 행마다 shuffle이 달라지도록
    for i, (c, a) in enumerate(zip(df5["choices"], df5["answer"]))
]

df5["new_choices"] = [c for c, _ in shuffled_df5]
df5["new_answer"]  = [a for _, a in shuffled_df5]


/tmp/ipykernel_1298516/1841385885.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5["new_choices"] = [c for c, _ in shuffled_df5]
/tmp/ipykernel_1298516/1841385885.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5["new_answer"]  = [a for _, a in shuffled_df5]


In [68]:
df5.new_answer.value_counts()

new_answer
5    262
2    255
3    248
4    246
1    228
Name: count, dtype: int64

In [12]:
check_df = df5.assign(
    original_answer_text = df5.apply(
        lambda r: r["choices"][r["answer"] - 1], axis=1
    ),
    shuffled_answer_text = df5.apply(
        lambda r: r["new_choices"][r["new_answer"] - 1], axis=1
    )
)

(check_df["original_answer_text"] == check_df["shuffled_answer_text"]).value_counts()

True    1239
Name: count, dtype: int64

In [13]:
df5.choices = df5.new_choices
df5.answer = df5.new_answer

/tmp/ipykernel_1298516/3925679504.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5.choices = df5.new_choices
/tmp/ipykernel_1298516/3925679504.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5.answer = df5.new_answer


In [14]:
df5 = df5[list(df4.columns)]

In [15]:
df5.columns

Index(['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus',
       'choices_len'],
      dtype='object')

In [16]:
df = pd.concat([df4,df5]).reset_index(drop = True)

In [17]:
train_df, valid_df = train_test_split(
    df,
    test_size=0.15,        
    random_state=42,
    stratify=df["choices_len"]
)

In [19]:
valid_df[valid_df.choices_len==4].answer.value_counts()

answer
4    35
1    34
2    26
3    24
Name: count, dtype: int64

In [32]:
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
valid_ds = Dataset.from_pandas(valid_df.reset_index(drop=True))

dataset = DatasetDict({
    "train": train_ds,
    "validation": valid_ds
})

In [33]:
MODEL_NAME = "Qwen/Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen3-8B"
    )
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-8B",
    torch_dtype=torch.float16,
    device_map="auto",
    )

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [34]:
### 정책별 System Prompt 함수
def get_system_message(row, system_prompts, prompt_policy):
    """
    row: 하나의 데이터 행
    system_prompts: {choices_len: {version: prompt_text}}
    prompt_policy: {choices_len: version}
    """
    choices_len = row["choices_len"]
    version = prompt_policy[choices_len]
    return system_prompts[choices_len][version]

In [35]:
# System Prompt 관리 구조 설명

# SYSTEM_PROMPTS:
# - 실제 system prompt 문구들을 모두 정의해 두는 "저장소"
# - 문제 유형(choices_len = 4 / 5)별로 나누고, 각 유형 안에서 v1, v2 처럼 여러 버전의 prompt를 보관
# - 즉, 여기에는 "쓸 수 있는 모든 프롬프트 후보"가 들어 있다.
#
# 예:
# SYSTEM_PROMPTS = {
#     4: { "v1": "...", "v2": "..." },
#     5: { "v1": "...", "v2": "..." }
# }

SYSTEM_PROMPT_4_V1 = (
    "당신은 **지식 추론(Knowledge Inference) 전문가**입니다. "
    "이 유형은 정답이 지문에 그대로 쓰여 있지 않을 수 있으며, 지문은 '조건/단서'를 제공합니다. "
    "지문에서 주어진 조건을 정확히 반영하고, 그 조건과 모순되지 않는 범위에서 일반적으로 알려진 지식을 적용해 "
    "가장 타당한 선택지 하나를 고르십시오."
)

SYSTEM_PROMPT_5_V1 = (
    "당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다. "
    "이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다. "
    "당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.\n\n"
)

# PROMPT_POLICY:
# - 이번 실험(run)에서 "어떤 system prompt 버전을 사용할지"를 결정하는 설정값
# - choices_len(4 or 5) → 사용할 prompt 버전(v1, v2, ...)
# - 실험을 바꿀 때는 이 딕셔너리만 수정

SYSTEM_PROMPTS = {
    4: {
        "v1": SYSTEM_PROMPT_4_V1,
    },
    5: {
        "v1": SYSTEM_PROMPT_5_V1,
    }
}

SYSTEM_PROMPT_POLICY = {
    4: "v1",
    5: "v1",
}

In [36]:
### 정책별 User Prompt 함수
def get_user_message(row, user_prompts, prompt_policy):
    """
    row: 데이터 행
    user_prompts: 템플릿 저장소
    prompt_policy: 버전 정책
    """
    # 메타 데이터 확인
    choices_len = row["choices_len"]
    version = prompt_policy[choices_len]
    
    # 해당 버전의 템플릿 세트 가져오기 (plus, no_plus가 들어있음)
    template_set = user_prompts[choices_len][version]
    
    # 데이터 준비
    paragraph = row['paragraph']
    question = row['question']
    choices_str = "\n".join([f"{i+1}. {c}" for i, c in enumerate(row['choices'])])
    q_plus = row.get('question_plus', None)
    
    # 분기 처리 및 포맷팅 (여기가 핵심!)
    # q_plus가 존재하고, nan이 아닐 때 -> Plus 템플릿 사용
    if q_plus and str(q_plus) != 'nan':
        return template_set["plus"].format(
            paragraph=paragraph,
            question_plus=q_plus, # 여기 들어감
            question=question,
            choices=choices_str
        )
    # q_plus가 없을 때 -> No Plus 템플릿 사용
    else:
        return template_set["no_plus"].format(
            paragraph=paragraph,
            question=question,
            choices=choices_str
        )

In [37]:
# =========================
# User Prompt Templates (V1)
# =========================

# 4지선다 + <보기> 있음
USER_PROMPT_PLUS_4_V1 = """### 지문
{paragraph}

### 질문
{question}

### 보기
{question_plus}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 지문이 주는 조건/단서를 먼저 정리하세요. (무엇을 가정/설명하고 있는지)
2. 필요하면 일반적으로 알려진 지식(개념/원리/사례)을 적용하되, 지문 조건과 모순되면 안 됩니다.
3. 선택지 중 조건을 가장 잘 만족하는 것 하나만 고르세요.

정답은 1~4 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""


# 4지선다 + <보기> 없음
USER_PROMPT_NO_PLUS_4_V1 = """### 지문
{paragraph}

### 질문
{question}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 지문이 주는 조건/단서를 먼저 정리하세요. (무엇을 가정/설명하고 있는지)
2. 필요하면 일반적으로 알려진 지식(개념/원리/사례)을 적용하되, 지문 조건과 모순되면 안 됩니다.
3. 선택지 중 조건을 가장 잘 만족하는 것 하나만 고르세요.

정답은 1~4 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""


# 5지선다 + <보기> 있음
USER_PROMPT_PLUS_5_V1 = """### 지문
{paragraph}

### 질문
{question}

### 보기
{question_plus}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 지문을 끝까지 읽고 핵심 정보를 정리하세요.
2. 질문이 요구하는 정보(수치/인물/원인/결과/요지 등)가 무엇인지 정확히 확인하세요.
3. 각 선택지가 지문의 어느 부분과 일치하는지 1:1로 대조하세요.
4. 지문과 모순되거나 지문에 근거가 없는 선택지는 제외하세요.
5. 가장 확실한 근거를 가진 선택지 번호 하나만 선택하세요.

정답은 1~5 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""


# 5지선다 + <보기> 없음
USER_PROMPT_NO_PLUS_5_V1 = """### 지문
{paragraph}

### 질문
{question}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 지문을 끝까지 읽고 핵심 정보를 정리하세요.
2. 질문이 요구하는 정보(수치/인물/원인/결과/요지 등)가 무엇인지 정확히 확인하세요.
3. 각 선택지가 지문의 어느 부분과 일치하는지 1:1로 대조하세요.
4. 지문과 모순되거나 지문에 근거가 없는 선택지는 제외하세요.
5. 가장 확실한 근거를 가진 선택지 번호 하나만 선택하세요.

정답은 1~5 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""

USER_PROMPTS = {
    4: {
        "v1": {
            "plus": USER_PROMPT_PLUS_4_V1,
            "no_plus": USER_PROMPT_NO_PLUS_4_V1,
        },
        # "v2": {...}
    },
    5: {
        "v1": {
            "plus": USER_PROMPT_PLUS_5_V1,
            "no_plus": USER_PROMPT_NO_PLUS_5_V1,
        },
        # "v2": {...}
    }
}

USER_PROMPT_POLICY = {
    4: "v1",
    5: "v1",
}

In [38]:
# 공통 Assistant Prompt 함수
def get_assistant_message(row):
    """
    Assistant 메시지 생성 함수.
    Qwen3 모델의 토크나이저 템플릿이 자동으로 <think> 태그를 처리하므로,
    여기서는 순수한 정답(Label) 텍스트만 반환
    """
    return str(row['answer'])

In [39]:
def build_messages(example):
    """
    원본 example(row)로부터 학습용 chat messages를 구성한다.
    - choices_len(4/5) 및 question_plus 유무에 따라 system/user 프롬프트를 선택
    - assistant는 정답 숫자만
    - 이후 평가/추적용으로 id, label도 함께 유지
    """
    sys_msg = get_system_message(example, SYSTEM_PROMPTS, SYSTEM_PROMPT_POLICY)
    user_msg = get_user_message(example, USER_PROMPTS, USER_PROMPT_POLICY)
    asst_msg = get_assistant_message(example)

    return {
        "id": example["id"],
        "messages": [
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": asst_msg},
        ],
        "label": int(example["answer"]),
    }

In [40]:
def to_text(example):
    """
    messages(list[dict])를 tokenizer의 chat_template 규칙에 따라
    단일 텍스트로 직렬화한다.
    """
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False,  
    )
    return {"text": text}

In [41]:
def tokenize_fn(example, truncation=True, max_length=2048, padding=False):
    """
    batched=True면 example["text"]는 List[str]
    batched=False면 example["text"]는 str
    """
    tok_kwargs = dict(truncation=truncation, padding=padding)
    if truncation is True:
        tok_kwargs["max_length"] = max_length

    out = tokenizer(example["text"], **tok_kwargs)
    
    return {
        "input_ids": out["input_ids"],
        "attention_mask": out["attention_mask"],
    }


In [42]:
orig_cols = dataset["train"].column_names
print(orig_cols)

['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus', 'choices_len']


In [43]:
dataset_msg = dataset.map(
    build_messages,
    batched=False,
    remove_columns=orig_cols,
    desc="Build messages",
)

Build messages:   0%|          | 0/1726 [00:00<?, ? examples/s]

Build messages:   0%|          | 0/305 [00:00<?, ? examples/s]

In [44]:
dataset_text = dataset_msg.map(
    to_text,
    batched=False,
    remove_columns=["messages"],
    desc="Serialize to text",
)

Serialize to text:   0%|          | 0/1726 [00:00<?, ? examples/s]

Serialize to text:   0%|          | 0/305 [00:00<?, ? examples/s]

In [45]:
tokenized_dataset = dataset_text.map(
    tokenize_fn,
    batched=True,
    num_proc=4, 
    remove_columns=["text"],
    load_from_cache_file=True,
    desc="Tokenizing",
)

Tokenizing (num_proc=4):   0%|          | 0/1726 [00:00<?, ? examples/s]

Tokenizing (num_proc=4):   0%|          | 0/305 [00:00<?, ? examples/s]

In [101]:
print("\n=== 변환 완료 ===")
print("Train 개수:", len(tokenized_dataset["train"]))
print("첫 번째 샘플 Keys:", tokenized_dataset["train"][0].keys())


=== 변환 완료 ===
Train 개수: 1726
첫 번째 샘플 Keys: dict_keys(['id', 'label', 'input_ids', 'attention_mask'])


In [46]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1726
    })
    validation: Dataset({
        features: ['id', 'label', 'input_ids', 'attention_mask'],
        num_rows: 305
    })
})

In [47]:
response_template = "<|im_start|>assistant\n"
data_collator = DataCollatorForCompletionOnlyLM(
    response_template=response_template,
    tokenizer=tokenizer,
)

In [48]:
### Qwen3 1,2,3,4,5 토큰 id 확인
# 가장 확실: encode 결과 보기
for s in ["1","2","3","4","5"," 1"," 2"," 3"," 4"," 5"]:
    ids = tokenizer.encode(s, add_special_tokens=False)
    toks = tokenizer.convert_ids_to_tokens(ids)
    print(f"{s!r} -> ids={ids}, toks={toks}")

# 단일 토큰 매핑을 기대한다면 (토크나이저에 따라 None/-1일 수 있음)
for tok in ["1","2","3","4","5"]:
    print(tok, "->", tokenizer.convert_tokens_to_ids(tok))

'1' -> ids=[16], toks=['1']
'2' -> ids=[17], toks=['2']
'3' -> ids=[18], toks=['3']
'4' -> ids=[19], toks=['4']
'5' -> ids=[20], toks=['5']
' 1' -> ids=[220, 16], toks=['Ġ', '1']
' 2' -> ids=[220, 17], toks=['Ġ', '2']
' 3' -> ids=[220, 18], toks=['Ġ', '3']
' 4' -> ids=[220, 19], toks=['Ġ', '4']
' 5' -> ids=[220, 20], toks=['Ġ', '5']
1 -> 16
2 -> 17
3 -> 18
4 -> 19
5 -> 20


In [49]:
### 검증용 / 미리 사전 체크
DIGIT_IDS = [16, 17, 18, 19, 20]  # '1'~'5'

def assert_answer_pos_is_digit(
    ds,                    # train_ds or eval_ds (collator 전에 remove_columns 한 ds)
    data_collator,         # DataCollatorForCompletionOnlyLM
    n_samples=1000,
    seed=42,
    pos_from_tail=3,       # -3 규칙
    batch_size=16,
    verbose_fail=5,
):
    rng = random.Random(seed)
    n_samples = min(n_samples, len(ds))
    idxs = rng.sample(range(len(ds)), n_samples)

    fail = 0
    shown = 0

    for start in range(0, n_samples, batch_size):
        batch_idxs = idxs[start:start+batch_size]
        sample_batch = [ds[i] for i in batch_idxs]
        collated = data_collator(sample_batch)

        input_ids = collated["input_ids"]          # (B, L)
        attn = collated["attention_mask"]          # (B, L)
        labels = collated["labels"]                # (B, L)

        real_len = attn.sum(dim=1)                 # (B,)
        pos = real_len - pos_from_tail             # (B,)

        bsz = input_ids.size(0)
        for b in range(bsz):
            p = int(pos[b].item())
            tok = int(input_ids[b, p].item())
            lab = int(labels[b, p].item())

            ok = (tok in DIGIT_IDS) and (lab in DIGIT_IDS)
            if not ok:
                fail += 1
                if shown < verbose_fail:
                    shown += 1
                    print(f"[FAIL] idx={batch_idxs[b]} real_len={int(real_len[b])} pos={p} tok={tok} lab={lab}")
                    # 주변 토큰도 같이 확인
                    left = max(0, p-6)
                    right = min(input_ids.size(1), p+6)
                    print("  decoded window:", tokenizer.decode(input_ids[b, left:right], skip_special_tokens=False))
                    # labels에서 digit 토큰이 아예 있는지 확인
                    digit_mask = torch.isin(labels[b], torch.tensor(DIGIT_IDS, device=labels.device))
                    if digit_mask.any():
                        digit_positions = torch.where(digit_mask)[0].tolist()
                        print("  digit_positions_in_labels:", digit_positions)
                    else:
                        print("  digit_positions_in_labels: NONE")

    assert fail == 0, f"pos=real_len-{pos_from_tail} 규칙이 깨진 샘플이 {fail}개 있습니다."
    print(f"OK: {n_samples} samples passed. (pos=real_len-{pos_from_tail} is digit token)")

In [50]:
DIGIT_IDS = [16, 17, 18, 19, 20]  # '1'~'5'

def preprocess_logits_for_metrics(logits, labels, pos_from_tail=4):
    """
    반환: (batch, 5)  -> '1'~'5'에 해당하는 logits만 뽑아서 metrics 단계로 전달
    """
    # Trainer가 (logits, ...) 튜플을 줄 때가 있어서 정리
    if isinstance(logits, tuple):
        logits = logits[0]  # (B, L, V)

    # labels: (B, L), pad/무시 영역은 -100일 가능성이 큼
    # real_len = 마지막으로 labels != -100 인 위치 + 1 로 복원
    labels_t = torch.as_tensor(labels)
    not_ignored = (labels_t != -100)

    # 샘플별로 마지막 not_ignored 위치 찾기
    # (뒤에서부터 True 찾기)
    rev = torch.flip(not_ignored, dims=[1])
    last_true_from_end = torch.argmax(rev.int(), dim=1)          # (B,)
    has_any = not_ignored.any(dim=1)                             # (B,)
    # real_len = seq_len - last_true_from_end
    seq_len = labels_t.size(1)
    real_len = seq_len - last_true_from_end

    # 만약 labels가 전부 -100인 샘플이 있으면(비정상) 그냥 seq_len로 처리
    real_len = torch.where(has_any, real_len, torch.full_like(real_len, seq_len))

    pos = (real_len - pos_from_tail).clamp(min=0, max=seq_len-1) # (B,)

    # (B, V)로 해당 위치의 logits만 gather
    logits_t = torch.as_tensor(logits)                           # (B, L, V)
    batch_idx = torch.arange(logits_t.size(0), device=logits_t.device)
    picked = logits_t[batch_idx, pos, :]                         # (B, V)

    # digit ids만 슬라이스 -> (B, 5)
    picked_digits = picked[:, DIGIT_IDS]
    return picked_digits

In [51]:
def compute_metrics(eval_pred, label_pos_from_tail=3):
    """
    eval_pred:
      - (predictions, label_ids) 튜플 형태가 가장 흔함
      - predictions: preprocess_logits_for_metrics가 반환한 (B, 5)
      - label_ids: (B, L) with -100 ignored
    반환: {"accuracy": ..., "macro_f1": ...}
    """
    if hasattr(eval_pred, "predictions"):
        preds, labels = eval_pred.predictions, eval_pred.label_ids
    else:
        preds, labels = eval_pred

    preds_t = torch.as_tensor(preds)
    pred_cls = torch.argmax(preds_t, dim=-1).cpu().numpy().astype(np.int64)  # (B,)

    labels_t = torch.as_tensor(labels)

    not_ignored = (labels_t != -100)
    rev = torch.flip(not_ignored, dims=[1])
    last_true_from_end = torch.argmax(rev.int(), dim=1)
    has_any = not_ignored.any(dim=1)

    seq_len = labels_t.size(1)
    real_len = seq_len - last_true_from_end
    real_len = torch.where(has_any, real_len, torch.full_like(real_len, seq_len))

    pos_label = (real_len - label_pos_from_tail).clamp(min=0, max=seq_len - 1)
    batch_idx = torch.arange(labels_t.size(0), device=labels_t.device)
    gold_tok = labels_t[batch_idx, pos_label].cpu().numpy().astype(np.int64) 

    gold_cls = gold_tok - DIGIT_IDS[0]  

    valid = (gold_cls >= 0) & (gold_cls < 5)
    pred_cls = pred_cls[valid]
    gold_cls = gold_cls[valid]

    acc = (pred_cls == gold_cls).mean() if len(gold_cls) > 0 else 0.0

    f1s = []
    for c in range(5):
        tp = np.sum((pred_cls == c) & (gold_cls == c))
        fp = np.sum((pred_cls == c) & (gold_cls != c))
        fn = np.sum((pred_cls != c) & (gold_cls == c))

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1        = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0
        f1s.append(f1)

    macro_f1 = float(np.mean(f1s)) if len(f1s) > 0 else 0.0

    return {"accuracy": float(acc), "macro_f1": macro_f1}

In [108]:
tokenizer.special_tokens_map

{'eos_token': '<|im_end|>',
 'pad_token': '<|endoftext|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

In [52]:
# Trainer 전에는 반드시 지워야 하는것 -> 'input_ids', 'attention_mask'
# 아래 예시처럼
train_dataset = tokenized_dataset["train"].remove_columns(["id", "label"])
eval_dataset = tokenized_dataset["validation"].remove_columns(["id", "label"])

In [53]:
print("pad_token:", tokenizer.pad_token, tokenizer.pad_token_id)
print("eos_token:", tokenizer.eos_token, tokenizer.eos_token_id)

# pad_token 없으면 eos로 대체
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

# 학습 안정/메모리 옵션
model.config.use_cache = False
model.gradient_checkpointing_enable()

print("use_cache:", model.config.use_cache)
print("grad_ckpt:", model.is_gradient_checkpointing)

pad_token: <|endoftext|> 151643
eos_token: <|im_end|> 151645
use_cache: False
grad_ckpt: True


In [54]:
MODEL_NAME = "Qwen/Qwen3-8B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,  # V100이면 fp16 추천
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)
model.config.use_cache = False
model.gradient_checkpointing_enable()

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [55]:
# tokenizer에 pad_token이 이미 있는 상태라면
model.config.pad_token_id = tokenizer.pad_token_id
model.generation_config.pad_token_id = tokenizer.pad_token_id

In [56]:
model = prepare_model_for_kbit_training(model)

# Attention proj만
# target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]

# Attention + MLP까지 (성능 더 노리되 trainable 조금 증가)
# "gate_proj", "up_proj", "down_proj" -> FFN
# target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

# target_modules = ["q_proj", "k_proj"]

target_modules = [
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj"
]

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 43,646,976 || all params: 8,234,382,336 || trainable%: 0.5301


In [57]:
training_args = SFTConfig(
    output_dir="../../qwen-sft-results",
    
    # 데이터 및 배치 설정
    num_train_epochs=1,
    max_seq_length=2048,
    packing=False,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    
    #  학습률 및 Optimizer (V100)
    learning_rate=2e-4,
    fp16=True,                  # V100 필수
    bf16=False,
    optim="paged_adamw_32bit",  #  32bit 사용 (안정성) / paged_adamw_8bit -> 더 안된다면 이걸로!
    gradient_checkpointing=True,
    
    # 검증(Eval) 및 저장(Save) 전략 수정
    eval_strategy="steps",      # Epoch 대신 Steps 단위로 검증
    eval_steps=20,              # 50 스텝마다 검증 (데이터 양에 따라 조절하세요)
    
    # 실제 사용 시 주석 해제
    save_strategy="steps",      # 검증 주기와 맞춰서 저장 전략도 steps로
    save_steps=20,              # 50 스텝마다 저장 시도
    
    # 저장 용량 관리
    save_total_limit=2,         # 체크포인트를 딱 2개만 남기고 옛날 건 자동 삭제!
    load_best_model_at_end=True, # 학습 끝나면 "가장 성능 좋았던 모델"을 자동으로 로드
    metric_for_best_model="eval_loss", # 무엇을 기준으로 최고를 뽑을지 (accuracy 추천)
    greater_is_better=False,
    
    # 기타 로깅
    logging_steps=10,
    report_to="none",

    # 시험용이라면???
    # max_steps = 50
)

In [58]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    args=training_args,
)

/data/ephemeral/pro-nlp-generationfornlp-nlp-13/.venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [59]:
torch.cuda.empty_cache()

trainer.train()


del model
del trainer
torch.cuda.empty_cache()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Step,Training Loss,Validation Loss,Accuracy,Macro F1
20,0.060000,0.068857,0.806557,0.815175
40,0.066100,0.059181,0.829508,0.838813
60,0.070500,0.058127,0.839344,0.849308
80,0.074300,0.059971,0.839344,0.851055
100,0.078800,0.059215,0.829508,0.838910
120,0.085300,0.064588,0.822951,0.836263
140,0.083200,0.058025,0.822951,0.832729
160,0.041300,0.056023,0.819672,0.830165
180,0.059500,0.060395,0.845902,0.854808
200,0.052700,0.059426,0.859016,0.867189


In [117]:
# 데이터셋 로드
test_df = pd.read_csv(os.path.join(DATA_DIR,'test.csv'))
test_df.info()

# Flatten the JSON dataset
records = []
for _, row in test_df.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)
        
# Convert to DataFrame
test_df = pd.DataFrame(records)
test_df["choices_len"] = test_df["choices"].apply(len)

test_dataset = Dataset.from_pandas(test_df)

# Model load
MODEL_NAME = "Qwen/Qwen3-8B"
adapter_path = "../../qwen-sft-results/checkpoint-216"# 실제 변경

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)

model = PeftModel.from_pretrained(base_model, adapter_path)

def build_test_messages(example):
    sys_msg = get_system_message(example, SYSTEM_PROMPTS, SYSTEM_PROMPT_POLICY)
    user_msg = get_user_message(example, USER_PROMPTS, USER_PROMPT_POLICY)

    return {
        "id": example["id"],
        "messages": [
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": user_msg},
        ]
    }

def to_test_text(example):
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=True, 
    )
    return {"text": text}

test_ds_msg = test_dataset.map(
    build_test_messages,
    batched=False,
    desc="Build messages",
)
test_ds_text = test_ds_msg.map(
    to_test_text,
    batched=False,
    desc="Serialize to text",
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 869 entries, 0 to 868
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     869 non-null    int64 
 1   id             869 non-null    object
 2   paragraph      869 non-null    object
 3   problems       869 non-null    object
 4   question_plus  44 non-null     object
dtypes: int64(1), object(4)
memory usage: 34.1+ KB


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Build messages:   0%|          | 0/869 [00:00<?, ? examples/s]

Serialize to text:   0%|          | 0/869 [00:00<?, ? examples/s]

In [118]:
### 이걸로 해보면...??? ###

DIGIT_IDS = [16, 17, 18, 19, 20]  # '1'~'5'
THINK_END_ID = 151668  # </think>

def get_answer_from_logits(outputs, input_len):
    """
    output_scores에서 </think> 이후 첫 digit의 logits 확인
    """
    if not hasattr(outputs, 'scores') or not outputs.scores:
        return None
    
    generated_ids = outputs.sequences[0]  # (total_len,)
    
    # </think> 위치 찾기
    think_end_positions = (generated_ids == THINK_END_ID).nonzero(as_tuple=True)[0]
    
    if len(think_end_positions) == 0:
        # </think>가 없으면 생성된 토큰 중 첫 digit 찾기
        for i in range(input_len, len(generated_ids)):
            token_id = generated_ids[i].item()
            if token_id in DIGIT_IDS:
                step_idx = i - input_len
                if 0 <= step_idx < len(outputs.scores):
                    step_logits = outputs.scores[step_idx][0]  # (V,)
                    digit_logits = step_logits[DIGIT_IDS]  # (5,)
                    return digit_logits.argmax().item() + 1  # 1~5
        return None
    
    # </think> 이후 첫 digit 토큰 찾기
    think_end_pos = think_end_positions[-1].item()
    
    for i in range(think_end_pos + 1, len(generated_ids)):
        token_id = generated_ids[i].item()
        if token_id in DIGIT_IDS:
            # 이 토큰이 생성된 step의 logits 확인
            step_idx = i - input_len
            if 0 <= step_idx < len(outputs.scores):
                step_logits = outputs.scores[step_idx][0]  # (V,)
                digit_logits = step_logits[DIGIT_IDS]  # (5,)
                # 가장 높은 확률의 digit 선택
                predicted = digit_logits.argmax().item() + 1  # 1~5
                return predicted
    
    return None


def parse_pred_fallback(text):
    """
    logits에서 실패 시 텍스트 파싱 (fallback)
    """
    if "</think>" in text:
        after_think = text.split("</think>")[-1]
        for char in after_think:
            if char in ['1', '2', '3', '4', '5']:
                return char
    
    for keyword in ["정답:", "정답은", "Answer:"]:
        if keyword in text:
            after_keyword = text.split(keyword)[-1]
            for char in after_keyword:
                if char in ['1', '2', '3', '4', '5']:
                    return char
    
    for char in reversed(text):
        if char in ['1', '2', '3', '4', '5']:
            return char
    
    return '1'


# ==========================================
# 추론 실행 (개선 버전)
# ==========================================
infer_results = []
logits_success = 0
fallback_used = 0

print("🚀 Logits 기반 추론 시작...")

model.eval()
with torch.inference_mode():
    for ex in tqdm(test_ds_text):
        _id = ex["id"]
        text = ex["text"]

        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=4096,
        ).to("cuda")

        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,
            temperature=0.0,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True,
            output_scores=True,       
        )

        input_len = inputs["input_ids"].shape[-1]
        
        # 1차 시도: logits 기반
        pred = get_answer_from_logits(outputs, input_len)
        
        if pred is not None:
            logits_success += 1
            pred_str = str(pred)
        else:
            # 2차 시도: 텍스트 파싱
            fallback_used += 1
            gen_ids = outputs.sequences[0][input_len:]
            gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True)
            pred_str = parse_pred_fallback(gen_text)
        
        infer_results.append({
            "id": _id,
            "answer": pred_str
        })

print(f"✅ 완료! 총 {len(infer_results)}개 결과 생성됨")
print(f"📊 Logits 성공: {logits_success} ({logits_success/len(infer_results)*100:.1f}%)")
print(f"📊 Fallback 사용: {fallback_used} ({fallback_used/len(infer_results)*100:.1f}%)")

🚀 Logits 기반 추론 시작...


100%|██████████| 869/869 [18:36<00:00,  1.28s/it]

✅ 완료! 총 869개 결과 생성됨
📊 Logits 성공: 869 (100.0%)
📊 Fallback 사용: 0 (0.0%)


In [120]:
pd.DataFrame(infer_results).to_csv('shuffled_answer.csv',index=False)